
### Price Trajectories
Modeled by [Wiener Process](https://en.wikipedia.org/wiki/Wiener_process)

### Data:
#### The Graph (Compound-v2)
Used for current total borrowed

#### Coingecko (Historical Prices)
Use file: `data/coingeckoe_prices.csv`

Created by download script `download_historical_prices.py`. The script has a runtime of ~1h, because it sends about 600 requests to coingecko. The free API allows 10-50 requests / minute, so I use a sleep of 6 seconds to never send more than 10 requests / minute. 

Time window: `2022-11-21` to `2022-11-20`

Should be updated if gets too old compared to the timestamp of data from The Graph. 


In [1]:
import pandas as pd
from pathlib import Path

from src.coingecko import CoingeckoAPI
from src.the_graph import TheGraphAPI

PRICES_FILE = Path.cwd().joinpath("data", "coingecko_prices.csv")

In [30]:
# get current Compound Borrows from The Graph

tg_api = TheGraphAPI()
borrows = tg_api.get_compound_borrows()

col_dtypes = {
    "underlyingSymbol":str,
    "underlyingName":str, 
    "borrowRate":float,	
    "totalBorrows":float,	
    "collateralFactor":float,	
    "underlyingPriceUSD":float,	
    "blockTimestamp":int,	
    "accrualBlockNumber":int
    }
borrows_df = pd.DataFrame(borrows)
borrows_df = borrows_df.apply(lambda c: c.astype(col_dtypes[c.name]))
borrows_df

,underlyingSymbol,underlyingName,borrowRate,totalBorrows,collateralFactor,underlyingPriceUSD,blockTimestamp,accrualBlockNumber
0,USDP,Pax Dollar,0.033180,1.973300e+04,0.000,1.000000,1669003091,16015853
1,TUSD,TrueUSD,0.032406,5.487731e+06,0.000,1.000000,1669059527,16020529
2,REP,Reputation,0.030407,5.590882e+01,0.000,6.961100,1667094731,15857708
3,UNI,Uniswap,0.041840,4.302136e+05,0.750,5.159703,1669058519,16019730
4,USDC,USD//C,0.018764,2.676037e+08,0.855,1.000000,1669059683,16020541
5,SUSHI,SushiToken,0.071078,6.520776e+05,0.700,1.111255,1669054991,16020155
6,ETH,Ether,0.022975,1.367234e+04,0.825,1080.714503,1669059551,16020529
7,DAI,Dai Stablecoin,0.023303,2.115244e+08,0.835,1.000938,1669059371,16020516
8,BAT,Basic Attention Token,0.020875,4.445730e+05,0.650,0.218834,1669057811,16020387
9,COMP,Compound,0.025741,1.024836e+04,0.650,36.916079,1669059371,16020516


Have two entries for `DAI` and `wBTC` each... only keep the bigger one for now
```
{
  "underlyingSymbol": "DAI",
  "underlyingName": "Dai Stablecoin",
  "borrowRate": "0.0233026004452608",
  "totalBorrows": "211524161.923877439677332104",
  "collateralFactor": "0.835",
  "underlyingPriceUSD": "1.000773",
  "blockTimestamp": 1669045535,
  "accrualBlockNumber": 16019369
},
{
  "underlyingSymbol": "DAI",
  "underlyingName": "Dai Stablecoin v1.0 (DAI)",
  "borrowRate": "0.0501467542606656",
  "totalBorrows": "240.64852988575940222",
  "collateralFactor": "0",
  "underlyingPriceUSD": "6.210826",
  "blockTimestamp": 1668979619,
  "accrualBlockNumber": 16013904
},
....
{
  "underlyingSymbol": "WBTC",
  "underlyingName": "Wrapped BTC",
  "borrowRate": "0.0206864728473312",
  "totalBorrows": "0.39387562",
  "collateralFactor": "0.7",
  "underlyingPriceUSD": "15929.996902",
  "blockTimestamp": 1669001867,
  "accrualBlockNumber": 16015751
},
{
  "underlyingSymbol": "WBTC",
  "underlyingName": "Wrapped BTC",
  "borrowRate": "0.0221761103560704",
  "totalBorrows": "152.21575285",
  "collateralFactor": "0.7",
  "underlyingPriceUSD": "16182.771321",
  "blockTimestamp": 1669043411,
  "accrualBlockNumber": 16019192
},
```

In [33]:
borrows_df = borrows_df.sort_values(by=["underlyingSymbol", "totalBorrows"], ascending=False).reset_index(drop=True)
borrows_df.drop_duplicates(subset=["underlyingSymbol"])

,underlyingSymbol,underlyingName,borrowRate,totalBorrows,collateralFactor,underlyingPriceUSD,blockTimestamp,accrualBlockNumber
0,ZRX,0x Protocol Token,0.032437,4.962050e+05,0.650,0.178897,1669057331,16020348
1,YFI,yearn.finance,0.020050,3.181422e-02,0.750,6194.460000,1668777299,15997132
2,WBTC,Wrapped BTC,0.022244,1.563179e+02,0.700,15722.936525,1669059323,16020512
4,USDT,Tether USD,0.033189,1.624361e+08,0.000,1.000000,1669054631,16020125
5,USDP,Pax Dollar,0.033180,1.973300e+04,0.000,1.000000,1669003091,16015853
6,USDC,USD//C,0.018764,2.676037e+08,0.855,1.000000,1669059683,16020541
7,UNI,Uniswap,0.041840,4.302136e+05,0.750,5.159703,1669058519,16019730
8,TUSD,TrueUSD,0.032406,5.487731e+06,0.000,1.000000,1669059527,16020529
9,SUSHI,SushiToken,0.071078,6.520776e+05,0.700,1.111255,1669054991,16020155
10,REP,Reputation,0.030407,5.590882e+01,0.000,6.961100,1667094731,15857708


In [2]:
from src.measures import calc_volatility, monte_carlo_simulation

# load historical prices from coingecko (used scrip `download_historica_prices.ipynb` to prepare data)

prices = pd.read_csv(PRICES_FILE)

vols = {}
for symbol in prices["symbol"].unique():
    p = prices[prices["symbol"] == symbol]["price"].values
    vols[symbol] = calc_volatility(p)

print(vols)

{'usdp': 0.0022709251521762157, 'tusd': 0.0006648082158006427, 'rep': 0.027848473702182807, 'uni': 0.030401066885438317, 'usdc': 0.0007862135517590391, 'sushi': 0.06101111212451682, 'eth': 0.037073182293776444, 'dai': 0.0005418670742110441, 'bat': 0.023631436362027365, 'comp': 0.032583239051199364, 'fei': 0.016986592573876688, 'yfi': 0.02476750122544395, 'zrx': 0.026416905420586858, 'wbtc': 0.01966328939099861, 'aave': 0.04395087279428308, 'usdt': 0.0003865638609175589, 'link': 0.027996528869455604}


In [54]:

mc = monte_carlo_simulation(start_price=15929.996902, volatility=vols["wbtc"], horizon=10, paths=4)
print(mc)


[[16193.91435662 16229.67487085 15996.51065905 16679.41981017
  16231.08714505 16178.69463361 16973.63370141 17582.54800306
  17737.81340256 17299.29782762]
 [16044.68848062 16137.89274497 16313.08318193 15963.22906553
  15587.58109322 15278.97233278 15301.79693468 15102.23548935
  15015.55611249 14918.96260072]
 [16394.70777922 16524.66157896 17133.40773836 18124.50344321
  17741.48913444 17649.1194142  17825.80000392 17833.63063382
  17503.00342352 17394.77133763]
 [15771.57325805 15799.21796918 16063.48049937 16094.84687627
  15956.68276504 16232.57032261 16264.75098504 16524.93181549
  16927.6982165  17317.79241789]]
